### Prematurity classification on the dHCP database.

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score, permutation_test_score

from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline, Pipeline
import re
import warnings
from dataclasses import dataclass
from typing import List, Optional, Tuple
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedGroupKFold, KFold
from sklearn.utils.validation import check_is_fitted
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
base_path = "/neurospin/dico/data/deep_folding/current/models/Champollion_V1_after_ablation"
labels_path = "/neurospin/dico/data/deep_folding/current/datasets/dHCP_374_subjects/participants.csv"

In [4]:
def _normalize_ids(s):

    s = s.astype(str).str.strip().str.lower()

    s = s.str.replace(r'^(sub-)', '', regex=True)
    return s


def load_cognition_df():
    path = "/neurospin/dico/rmenasria/Runs/03_main/Input/dHCP/cognitive_scores_with_age_dHCP.csv"
    df = pd.read_csv(path)
    if "Participant ID" not in df.columns:
        raise ValueError("Colonne 'Participant ID' absente du CSV cognition.")
    if "Child's Sex" not in df.columns:
        raise ValueError("Colonne \"Child's Sex\" absente du CSV cognition.")
    df["pid_norm"] = _normalize_ids(df["Participant ID"])
    df = df.drop_duplicates(subset=["pid_norm"], keep="first")
    return df[["pid_norm", "Child's Sex"]]


def load_participants_labels(labels_path):
    df = pd.read_csv(labels_path)
    if "Subject" not in df.columns or "birth_age" not in df.columns:
        raise ValueError("Le CSV participants doit contenir 'Subject' et 'birth_age'.")
    df["pid_norm"] = _normalize_ids(df["Subject"])
    # fabriquer prem_class depuis birth_age
    ba = df["birth_age"].astype(float)
    prem_class = np.where(ba < 28, "<28",
                   np.where(ba < 32, "28-32",
                   np.where(ba < 37, "32-37", ">=37")))
    df["prem_class"] = prem_class
    df = df.drop_duplicates(subset=["pid_norm"], keep="first")
    return df[["pid_norm", "prem_class"]]


def load_embeddings_dHCP(region) :
    region_path = os.path.join(base_path, region)
    subdirs = [d for d in os.listdir(region_path) if os.path.isdir(os.path.join(region_path, d))]
    if len(subdirs) != 1:
        raise RuntimeError(f"{region_path}: attendu 1 sous-dossier de modèle, trouvé {len(subdirs)}.")
    model_folder = subdirs[0]
    embedding_path = os.path.join(region_path, model_folder, "dHCP_random_embeddings", "full_embeddings.csv")
    emb = pd.read_csv(embedding_path, index_col=0)
    emb.index = _normalize_ids(emb.index)
    emb.index.name = "pid_norm"
    emb = emb[~emb.index.duplicated(keep="first")]
    return emb

In [5]:
class ResidualizerSexeFromX(BaseEstimator, TransformerMixin):
    """
    Suppose que la dernière colonne de X est le sexe binaire {0,1}.
    - fit : pour chaque feature j, ajuste x_j ~ a_j + b_j*sexe
    - transform : retourne les résidus (x_j - prédiction), en enlevant la colonne sexe.
    """
    def __init__(self, sex_col_index: int = -1):
        self.sex_col_index = sex_col_index

    def fit(self, X, y=None):
        
        X = np.asarray(X, dtype=float)
        sex = X[:, self.sex_col_index]
        feats = np.delete(X, self.sex_col_index, axis=1)

        # Vérif binaire
        uniq = np.unique(sex[~np.isnan(sex)])
        if not np.all(np.isin(uniq, [0.0, 1.0])):
            warnings.warn("Colonne sexe pas strictement binaire {0,1}.")

        # Formules fermées de la régression OLS par dimension
        s_mean, s_var = np.nanmean(sex), np.nanvar(sex)
        X_mean = np.nanmean(feats, axis=0)

        if s_var <= 0:
            beta = np.zeros(feats.shape[1])
            alpha = X_mean.copy()
        else:
            cov = np.nanmean((feats - X_mean) * (sex[:, None] - s_mean), axis=0)
            beta = cov / s_var
            alpha = X_mean - beta * s_mean

        self.alpha_ = alpha
        self.beta_ = beta
        self.n_features_ = feats.shape[1]
        self.sex_col_index_ = self.sex_col_index
        return self

    def transform(self, X):
        check_is_fitted(self, ["alpha_", "beta_"])
        X = np.asarray(X, dtype=float)
        sex = X[:, self.sex_col_index_]
        feats = np.delete(X, self.sex_col_index_, axis=1)
        pred = self.alpha_[None, :] + sex[:, None] * self.beta_[None, :]

        return feats - pred

In [6]:
@dataclass
class PreparedData:
    X: np.ndarray            # embeddings + sexe concaténés (dernière colonne = sexe)
    y: np.ndarray            # binaire: 1 = prem_target, 0 = autre (dans prem_class filtrées)
    groups: np.ndarray
    idx_keep: np.ndarray
    embedding_cols: List[str]  # noms des features + ["_sex_num"] à la fin
    n_splits: int

def prepare_dhcp_for_prematurity(
    region: str,
    labels_path: str,
    prem_class: List[str],       # p.ex. ["28-32", ">=37"]
    prem_target: str,            # p.ex. "28-32"
    sex_col: str = "Child's Sex",
    group_col: Optional[str] = None,
    min_splits: int = 5
) -> PreparedData:
    emb_df = load_embeddings_dHCP(region)
    part_df = load_participants_labels(labels_path)      # prem_class depuis birth_age
    sex_df = load_cognition_df()                         # sexe depuis cognition

    # merge Embeddings × Participants (labels)
    merged = part_df.merge(emb_df, left_on="pid_norm", right_index=True, how="inner")

    # merge sexe
    merged = merged.merge(sex_df, on="pid_norm", how="left")
    if merged["Child's Sex"].isna().any():
        n_missing = merged["Child's Sex"].isna().sum()
        warnings.warn(f"{region}: {n_missing} sujets sans sexe dans le CSV cognition — ils seront droppés.")

    # filtre classes gardées
    before = merged.shape[0]
    merged = merged[merged["prem_class"].isin(prem_class)].copy()
    print(f"[{region}] filtre prem_class {prem_class}: {before} -> {merged.shape[0]}")

    # encode sexe
    sex_map = {"male": 0, "female": 1, "m": 0, "f": 1}
    merged["_sex_num"] = _normalize_ids(merged[sex_col]).map(sex_map).astype(float)

    # features embedding
    embedding_cols = [c for c in merged.columns if re.match(r"^dim", c)]
    if not embedding_cols:
        raise ValueError("Aucune colonne d'embeddings 'dim...' trouvée après merge.")
    X_emb = merged[embedding_cols].values.astype(float)

    # target binaire
    y = (merged["prem_class"] == prem_target).astype(int).values
    sex_vec = merged["_sex_num"].values.astype(float)

    # groups (facultatif)
    if group_col is None:
        groups = merged["pid_norm"].astype(str).values
    else:
        if group_col not in merged.columns:
            raise ValueError(f"Colonne de groupe '{group_col}' absente du merged.")
        groups = merged[group_col].astype(str).values

    # clean
    keep = (~pd.isna(y)) & (~np.isnan(sex_vec))
    X_emb, y, sex_vec, groups = X_emb[keep], y[keep], sex_vec[keep], groups[keep]

    # concat sexe en dernière colonne
    X_aug = np.column_stack([X_emb, sex_vec])

    # n_splits selon minorité
    counts = pd.Series(y).value_counts()
    max_splits = int(counts.min()) if not counts.empty else 2
    n_splits = max(2, min(min_splits, max_splits))

    return PreparedData(
        X=X_aug, y=y, groups=groups, idx_keep=keep,
        embedding_cols=embedding_cols + ["_sex_num"], n_splits=n_splits
    )


In [7]:
def classify_prematurity_with_perm(
    data: PreparedData,
    Cs: Tuple[float, ...] = (0.01, 0.1, 1, 10),
    n_splits: Optional[int] = None,
    random_state: int = 42,
    n_jobs: int = -1,
    n_permutations: int = 200
):
    if n_splits is None:
        n_splits = data.n_splits

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    pipe = Pipeline([
        ("resid", ResidualizerSexeFromX()),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(class_weight="balanced", solver="liblinear"))
    ])

    # sélection de C par CV
    results = []
    for C in Cs:
        pipe.set_params(clf__C=C)
        aucs = cross_val_score(
            pipe, data.X, data.y,
            cv=skf, scoring="roc_auc", n_jobs=n_jobs
        )
        results.append((C, aucs))
    best_idx = int(np.argmax([np.mean(aucs) for _, aucs in results]))
    best_C, best_aucs = results[best_idx]

    # permutations uniquement sur best_C
    pipe.set_params(clf__C=best_C)
    perm_score, perm_scores, pval = permutation_test_score(
        pipe, data.X, data.y,
        scoring="roc_auc", cv=skf,
        n_permutations=n_permutations, n_jobs=n_jobs,
        random_state=random_state
    )
    ci95 = float(np.percentile(perm_scores, 95))

    # fit final pour coefficients (dans l'espace original, après résidualisation, avant scaling)
    pipe.fit(data.X, data.y)
    scaler = pipe.named_steps["scaler"]
    clf = pipe.named_steps["clf"]
    coefs_scaled = clf.coef_.ravel()
    coefs_unscaled = coefs_scaled / scaler.scale_
    intercept_unscaled = float(clf.intercept_ - np.sum(coefs_scaled * scaler.mean_ / scaler.scale_))
    coef_dict = {name: float(w) for name, w in zip(data.embedding_cols, coefs_unscaled)}

    return {
        "best_C": float(best_C),
        "AUC_mean": float(np.mean(best_aucs)),
        "AUC_std": float(np.std(best_aucs)),
        "perm_score": float(perm_score),
        "perm_pval": float(pval),
        "perm_ci95": ci95,
        "all_results": {float(C): aucs.tolist() for C, aucs in results},
        "coef_unscaled": coef_dict,
        "intercept_unscaled": intercept_unscaled
    }

In [18]:
def prem_direction(data):
    # direction prematurité
    pipe = Pipeline([
        ("resid", ResidualizerSexeFromX()),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(class_weight="balanced", C=10, solver="liblinear"))
    ])
    pipe.fit(data.X, data.y)
    pipe.predict(data.X)

    # get the subject names and their corresponding predicted scores

    # Projection brute des individus
    projection = pipe.predict_proba(data.X)[:, 1]

    df = pd.DataFrame({
        "subject": data.groups,   # identifiant sujet
        "y_true": data.y,         # score cognitif
        "projection": projection  # projection sur la direction
    })

    # trier du plus grand au plus petit
    df = df.sort_values(by="projection", ascending=False).reset_index(drop=True)
    return df




In [19]:
data = prepare_dhcp_for_prematurity(
    "FCMpost-SpC_right",
    labels_path=labels_path,
    prem_class=['28-32','>=37'],
    prem_target='28-32',
)

df_proj = prem_direction(data)

df_proj.to_csv(
    "/neurospin/dico/rmenasria/Runs/03_main/Output/final_direction/dHCP/projection_FCMpost-SpC_right_28_32.csv",
    index=False)

[FCMpost-SpC_right] filtre prem_class ['28-32', '>=37']: 374 -> 312


In [22]:
# extract top and bottom 12 in upper case

df_proj = pd.read_csv(
    "/neurospin/dico/rmenasria/Runs/03_main/Output/final_direction/dHCP/projection_STs_right_28_32.csv"
)
top12 = df_proj.head(12).copy()["subject"].str.upper()
bottom12 = df_proj.tail(12).copy()["subject"].str.upper()

print("Top 12 subjects with highest projection scores:")
print(top12)
print("\nBottom 12 subjects with lowest projection scores:")
print(bottom12)



Top 12 subjects with highest projection scores:
0     CC00657XX14
1     CC00600XX06
2     CC00583XX15
3     CC00412XX08
4     CC00245BN15
5     CC00202XX04
6     CC00152AN04
7     CC00672AN13
8     CC00102XX03
9     CC00731XX14
10    CC00162XX06
11    CC00617XX15
Name: subject, dtype: object

Bottom 12 subjects with lowest projection scores:
293    CC00669XX18
294    CC00852XX11
295    CC00584XX16
296    CC00080XX07
297    CC01199XX21
298    CC00139XX16
299    CC00688XX21
300    CC00223XX09
301    CC00380XX10
302    CC00861XX12
303    CC00292XX13
304    CC01194XX16
Name: subject, dtype: object


In [59]:
def run_regions_prematurity(
    regions: List[str],
    labels_path: str,
    prem_class: List[str],
    prem_target: str,
    Cs=(0.01, 0.1, 1, 10),
    n_perm: int = 200,
    random_state: int = 42,
    n_jobs: int = -1,
    output_csv: Optional[str] = None,
    output_coef_csv: Optional[str] = None,  
) -> pd.DataFrame:
    rows = []
    coef_rows = []

    for region in regions:
        try:
            data = prepare_dhcp_for_prematurity(
                region=region,
                labels_path=labels_path,
                prem_class=prem_class,
                prem_target=prem_target
            )
            res = classify_prematurity_with_perm(
                data, Cs=Cs, n_permutations=n_perm,
                random_state=random_state, n_jobs=n_jobs
            )

            # résumé perf
            rows.append({
                "region": region,
                "n": int(len(data.y)),
                "best_C": res["best_C"],
                "AUC_mean": res["AUC_mean"],
                "AUC_std": res["AUC_std"],
                "perm_score": res["perm_score"],
                "perm_pval": res["perm_pval"],
                "perm_ci95": res["perm_ci95"]
            })

            # direction de classif (coefficients non-scalés)
            coef_dict = res["coef_unscaled"].copy()
            coef_dict.pop("_sex_num", None)  # on enlève la colonne sexe
            # récupérer les coeffs dans l'ordre des embeddings
            ordered_coeffs = [coef_dict[c] for c in data.embedding_cols if c != "_sex_num"]

            coef_rows.append({
                "region": region,
                "intercept": res["intercept_unscaled"],
                "coeffs": ordered_coeffs  # une seule colonne avec la liste
            })

        except Exception as e:
            warnings.warn(f"[{region}] échec: {e}")

    # CSV 1: résumé perf
    df = pd.DataFrame(rows)
    if output_csv:
        df.to_csv(output_csv, index=False)

    # CSV 2: directions
    if coef_rows:
        coef_df = pd.DataFrame(coef_rows)
        if output_coef_csv:
            # par défaut, pandas écrit la liste comme string JSON-like
            coef_df.to_csv(output_coef_csv, index=False)

    return df



In [55]:
def get_region_list(base_path):
    return sorted([
        d for d in os.listdir(base_path)
        if os.path.isdir(os.path.join(base_path, d))
           and not d.startswith('all_models')
           and not d.startswith('hcp')
           and not d.startswith('ukb')
           and not d.endswith('.csv')
           and not d.endswith('.sh')
           and not d.endswith('embeddings')
    ])

In [62]:
regions_test= [
    "SFinf-BROCA-SPeCinf_right",
    "SFinf-BROCA-SPeCinf_left",
    "ScCal-SLi_left"
]

regions = get_region_list(base_path)

prem_filter = ["<28", ">=37"]   # on garde uniquement ces classes
prem_target = "<28"             # la classe positive = prématurés

out = run_regions_prematurity(
    regions=regions,
    labels_path=labels_path,        
    prem_class=prem_filter,
    prem_target=prem_target,
    Cs=[0.01, 0.1, 1, 10],
    n_perm=11200,                     
    n_jobs=-1,
    output_csv="/neurospin/dico/rmenasria/Runs/03_main/Output/final/classif_prematurity_dHCP_final_28.csv",
    output_coef_csv="/neurospin/dico/rmenasria/Runs/03_main/Output/final/classif_prematurity_dHCP_directions_final_28.csv"
)
print(out)


[CINGULATE_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[CINGULATE_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[FCLp-subsc-FCLa-INSULA_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[FCLp-subsc-FCLa-INSULA_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[FCMpost-SpC_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[FCMpost-SpC_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[FColl-SRh_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[FColl-SRh_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[FIP_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[FIP_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[FPO-SCu-ScCal_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[FPO-SCu-ScCal_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[LARGE_CINGULATE_left] filtre prem_class ['<28', '>=37']: 374 -> 296
[LARGE_CINGULATE_right] filtre prem_class ['<28', '>=37']: 374 -> 296
[Lobule_parietal_sup_left] filtre prem_class ['<28', '>=37']: 374 -> 296


In [8]:
df = pd.read_csv('/neurospin/dico/rmenasria/Runs/01_essai/Program/2023_jlaval_STSbabies/contrastive/notebooks/racim/prematurity_classification_stats_1205.csv')

# Pivot pour avoir une colonne AUC par tranche
df_pivot = df.pivot(index='region', columns='tranche', values='AUC_mean')

# Renommer les colonnes pour clarifier
df_pivot = df_pivot.rename(columns={
    '<27': 'AUC_27',
    '27-32': 'AUC_27_32',
    '32-37': 'AUC_32_37'
}).reset_index()

# Sauvegarder le nouveau CSV
output_csv = 'prematurity_AUC_by_region_1205.csv'
df_pivot.to_csv(output_csv, index=False)
print(f"Fichier généré : {output_csv}")





Fichier généré : prematurity_AUC_by_region_1205.csv


In [24]:
df_results = pd.read_csv("/neurospin/dico/rmenasria/Runs/03_main/Output/final/classif_prematurity_dHCP_final_28_32.csv")

# add a column thresholded_auc = AUC_mean if perm_pval < 0.05/56 else 0
df_results['thresholded_auc'] = df_results.apply(
    lambda row: row['AUC_mean'] if row['perm_pval'] < 0.05 / 56 else 0,
    axis=1
)

# Select the columns "region" and "thresholded_auc" and save to CSV
df_results[["region", "thresholded_auc"]].to_csv(
    "/neurospin/dico/rmenasria/Runs/03_main/Output/final/whole_brain_visu/prematurity_thresholded_auc_28_32.csv", 
    index=False
)


In [26]:
df_results_ABCD = pd.read_csv("/neurospin/dico/rmenasria/Runs/03_main/Output/final/prematurity/report_final/ABCD_prematurity_results_final_28_32_corrected.csv")

# only select columns region, AUC_mean, AUC_std, perm_pval
df_results_ABCD = df_results_ABCD[["region", "auc_mean", "auc_std", "perm_pvalue"]]
df_results_ABCD.to_csv(
    "/neurospin/dico/rmenasria/Runs/03_main/Output/final/prematurity/report_final/table_supplementary_ABCD_28_32.csv", 
    index=False
)

In [24]:
# Charger le fichier CSV d'origine
df = pd.read_csv('/neurospin/dico/rmenasria/Runs/03_main/Program/2025_rmenasria_prematurity/notebooks/racim/prematurity_classification_dHCP_0708.csv')

# Remplacer les AUC_mean non significatifs (p_value >= 0.05) par 0
df.loc[df['p_value'] >= 0.05, 'AUC_mean'] = 0.0

# Pivot pour avoir une colonne AUC par tranche
df_pivot = df.pivot(index='region', columns='tranche', values='AUC_mean')

# Renommer les colonnes
df_pivot = df_pivot.rename(columns={
    '<28': 'AUC_28',
    '28-32': 'AUC_28_32',
    '32-37': 'AUC_37'
}).reset_index()

# Exporter le résultat
output_csv = 'prematurity_AUC_by_region_thresholded_0708_dHCP.csv'
df_pivot.to_csv(output_csv, index=False)
print(f"Fichier généré : {output_csv}")


Fichier généré : prematurity_AUC_by_region_thresholded_0708_dHCP.csv
